## 数据合并需求

在实际的业务中，需要将多个文档、数据，可能是 Series 或 DataFrame 拼合在一起，进行大数据分析。Pandas 提供的各种功能轻而易举地进行这些工作。

应用场景
在作者来看，在以下场景下可能会需要对数据进行拼接、合并操作：

日常工作中由多方交付的表格要合并成一个总表格
从数据库导出数据，单次太大，多次导出后需要合并
有多个数据，需要制取各自中有用的信息组成一个新的表
等等
操作
数据的合并其实有合并、连接、拼接等几种，最简单的是连接。连接字段相同只是把新的内容追加在后边。合并可能是不同的列，把这些列组合在一起形成多个列。还有一种是混合的，在以上的基础上，合并过程中还需要做些计算。



## 数据准备

In [19]:
import pandas as pd

In [37]:
# 以下有三个 df ，后续的操作我们将对他们进行合并处理。

df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                   index=[0, 1, 2, 3])


df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                   index=[4, 5, 6, 7])


df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                   index=[8, 9, 10, 11])

df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                   'D': ['D2', 'D3', 'D6', 'D7'],
                   'F': ['F2', 'F3', 'F6', 'F7']},
                  index = [2,3,6,7])

s1 = pd.Series(['X0', 'X1', 'X2', 'X3'], name='X')

In [21]:
type(df1)  # df1是DataFrame

pandas.core.frame.DataFrame

## concat()

### DataFrame合并

#### df取并集

##### df列相同进行连接

In [48]:

frames = [df1, df2, df3] 
frames
type(frames)  # frames 是 list

df = pd.concat(frames) 
df = pd.concat([df1, df2, df3]) # 将三个有相同列的表合并到一起，并使用新的自然索引
df
type(df)  # df是DataFrame

df = pd.concat(frames, keys=['x','y','z']) 
df = pd.concat([df1,df2,df3], keys = ['x','y','z']) # keys= 给每个表给一个一级索引，形成多层索引
df = pd.concat({'x':df1, 'y':df2, 'z':df3}) # 使用字典，进行索引
df


A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

In [49]:
df = pd.concat([df1,df4], ignore_index=True, sort=False)

##### df列不同进行连接

In [33]:
df = pd.concat([df1,df4], axis=1, sort=False)  # df1 列为ABCD，df4列为BDF， 结果列为 ABCDBDF
df = pd.concat([df1,df4], axis=1, join='outer', sort=False)  # join = 'outer',表示取并集

In [34]:
df

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


#### df取交集

In [35]:
#  
df = pd.concat([df1, df4], axis=1, join='inner') # join = 'inner'表示取并集,只将有共同索引的内容进行了合并。

In [36]:
df

,A,B,C,D,B,D,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


#### df只取指定索引内容

In [ ]:
# 如果我们只需要第一张表索引的内容，可以：

# 以下两个方法效果一样
pd.concat([df1, df4], axis=1).reindex(df1.index)
pd.concat([df1, df4.reindex(df1.index)], axis=1)

### Series和DataFrame合并

In [40]:
df = pd.concat([df1, s1], axis=1)  
df = pd.concat([df1, s1], axis=1, ignore_index=True)  # ignore_index会取消列名，变成0，1，2，3，4

In [41]:
df

,0,1,2,3,4
0,A0,B0,C0,D0,X0
1,A1,B1,C1,D1,X1
2,A2,B2,C2,D2,X2
3,A3,B3,C3,D3,X3


### Series合并

In [45]:
# 多个Series合并为一个DataFrame
s3 = pd.Series([0, 1, 2, 3], name='foo') # 有列名
s4 = pd.Series([0, 1, 2, 3]) # 无列名
s5 = pd.Series([0, 1, 4, 5]) # 无列名

pd.concat([s3,s4,s5],axis=1) # 
pd.concat([s3,s4,s5],axis=1,keys=['red','yellow','blue']) # keys=, 赋予新的列名

,red,yellow,blue
0,0,0,0
1,1,1,1
2,2,2,4
3,3,3,5


## merge()

In [50]:
# Pandas 提供了 merge() 方法，具有全功能、高性能的内存连接操作，与 SQL 等关系数据库非常相似。
# 这些方法的性能比其他开源实现（如 R 语言中的 base::merge.data.frame）要好得多（在某些情况下甚至超过一个数量级）。

### 语法结构

In [51]:
# 可以将两个 DataFrame 或者 Series 进行连接：

# pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
#          left_index=False, right_index=False, sort=True,
#          suffixes=('_x', '_y'), copy=True, indicator=False,
#          validate=None)
# 可以实现类似 SQL 的 join 操作，参数为：

# how：连接方式，默认为inner，可设为inner/outer/left/right
# on：根据某个字段进行连接，必须存在于两个DateFrame中（若未同时存在，则需要分别使用left_on 和 right_on 来设置）
# left_on：左连接，以DataFrame1中用作连接键的列
# right_on：右连接，以DataFrame2中用作连接键的列
# left_index：bool, default False，将DataFrame1行索引用作连接键
# right_index：bool, default False，将DataFrame2行索引用作连接键
# sort：根据连接键对合并后的数据进行排列，默认为True
# suffixes：对两个数据集中出现的重复列，新数据集中加上后缀 _x, _y 进行区别

#### how

In [ ]:

# how 参数可以指定数据用哪种方法进行合并，没有的内容会为 NaN：

# 以左为表基表
pd.merge(left, right, how='left', on=['key1', 'key2'])

# 以右为表基表
pd.merge(left, right, how='right', on=['key1', 'key2'])

# 取两个表的合集
pd.merge(left, right, how='outer', on=['key1', 'key2'])

# 取两个表的交集
pd.merge(left, right, how='inner', on=['key1', 'key2'])


# 下边是一个有重复连接键的例子：

left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 2, 2]})
pd.merge(left, right, on='B', how='outer')

#### validate

In [ ]:
# 检查重复键 validate

# 一对多连接
left = pd.DataFrame({'A' : [1,2], 'B' : [1, 2]})
right = pd.DataFrame({'A' : [4,5,6], 'B': [2, 2, 2]})
pd.merge(left, right, on='B', how='outer', validate="one_to_many")

# 其他的还有：

# 如果指定，则检查合并是否为指定的类型：
# “one_to_one” 或 “1:1”: 检查合并键在左右数据集中是否唯一
# “one_to_many” 或 “1:m”: 检查合并键在左数据集中是否唯一
# “many_to_one” 或 “m:1”: 检查合并键在右数据集中是否唯一
# “many_to_many” 或 “m:m”: 允许，但不会检查

#### indicator

In [ ]:
# 连接指示 连接指示 indicator
# 如果设置 indicator 为 True, 则会增加名为 _merge 的一列，显示这列是多何而来，_merge 列有取三个值：

# left_only 只在左表中
# right_only 只在右表中
# both 两个表中都有
pd.merge(left, right, on='B', how='outer', indicator=True)

'''
   A_x  B  A_y     _merge
0    1  1  NaN  left_only
1    2  2  4.0       both
2    2  2  5.0       both
3    2  2  6.0       both
'''

### 单个连接键

In [69]:
# 数据准备
AB = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

CD = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

In [71]:
# pd.merge(AB,CD) 或者 AB.merge(CD)
pd.merge(AB,CD,how='left',on='key')  # on 为两个数据的连接键，都以 key 为标准。
AB.merge(CD, how='left', on='key')  

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


### 多个连接键

In [64]:
# 数据准备
AB = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

CD = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

In [66]:
pd.merge(AB,CD,how='inner',on=['key1','key2'])  # key1 和 key2 组合在两个表里共有的会被连接，两个表都有 (k1, k0) ，右表有两个，左表有一个，合并后右表共同了左表的内容。


,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


## pd.merge_ordered()

In [72]:
# pd.merge_ordered 函数允许组合时间序列和其他有序数据。 
# 特别是，它具有可选的fill_method 关键字来填充/插入丢失的数据：

In [73]:
# 数据准备
left = pd.DataFrame({'k': ['K0', 'K1', 'K1', 'K2'],
                     'lv': [1, 2, 3, 4],
                     's': ['a', 'b', 'c', 'd']})

right = pd.DataFrame({'k': ['K1', 'K2', 'K4'],
                      'rv': [1, 2, 3]})



In [ ]:
pd.merge_ordered(left, right, fill_method='ffill', left_by='s')  # 按s列排序

## df.combine_first()

In [74]:
# 在数据合并过程中需要对应位置的数值进行计算，比如相加、平均，对空值补齐等，
# Pandas 提供了 df.combine_first() 和 df. combine() 等方法进行这些操作。

In [ ]:
# df.combine_first()
# 使用相同位置的值更新空元素，它只能是 df1 有空元素时才能被替换，如果数据结构不一致，所得 DataFram e的行索引和列索引将是两者的并集。

df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine_first(df2)
'''
     A    B
0  1.0  3.0
1  0.0  4.0
'''
# 在上例中，df1 中的 A 和 B 的空值被 df2 中的相同位置值替换。

In [ ]:
# df. combine()
# 可以与另一个 DataFrame 进行按列组合。使用函数将一个 DataFrame 与其他DataFrame合并，以逐元素合并列。 所得 DataFrame 的行索引和列索引将是两者的并集。

# 这个函数中有两个参数，分别是两个 df 中对应的 series， 计算后返回一个 Series 或者标量。

df1 = pd.DataFrame({'A': [0, 0], 'B': [4, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
# s1 列总和如果小于 s2列总和取 s1, 否则取 s2
take_smaller = lambda s1, s2: s1 if s1.sum() < s2.sum() else s2
df1.combine(df2, take_smaller)
'''
A  B
0  0  3
1  0  3
'''

In [ ]:
# df.update()
# 使用来自另一个 DataFrame 的非NA值进行修改，原 df 为被更新。

df = pd.DataFrame({'A': [1, 2, 3],
                   'B': [400, 500, 600]})
new_df = pd.DataFrame({'B': [4, 5, 6],
                       'C': [7, 8, 9]})
df.update(new_df)
df
   A  B
0  1  4
1  2  5
2  3  6
# DataFrame的长度不会增加，只会更新匹配的索引/列标签上的值。

df = pd.DataFrame({'A': ['a', 'b', 'c'],
                   'B': ['x', 'y', 'z']})
new_df = pd.DataFrame({'B': ['d', 'e', 'f', 'g', 'h', 'i']})
df.update(new_df)
df
   A  B
0  a  d
1  b  e
2  c  f
# 对于系列，必须设置其名称属性。

df = pd.DataFrame({'A': ['a', 'b', 'c'],
                   'B': ['x', 'y', 'z']})
new_column = pd.Series(['d', 'e'], name='B', index=[0, 2])
df.update(new_column)
df
   A  B
0  a  d
1  b  y
2  c  e


df = pd.DataFrame({'A': ['a', 'b', 'c'],
                   'B': ['x', 'y', 'z']})
new_df = pd.DataFrame({'B': ['d', 'e']}, index=[1, 2])
df.update(new_df)
df
   A  B
0  a  x
1  b  d
2  c  e
# 如果其他包含NaN，则相应的值不会在原始数据帧中更新。

df = pd.DataFrame({'A': [1, 2, 3],
                   'B': [400, 500, 600]})
new_df = pd.DataFrame({'B': [4, np.nan, 6]})
df.update(new_df)
df
   A      B
0  1    4.0
1  2  500.0
2  3    6.0

## 数据对比

In [ ]:
# df.compare() 和s.compare() 方法使您可以分别比较两个DataFrame 或 Series


### compare()

#### 语法

In [ ]:
pd.compare(other, align_axis=1, keep_shape=False, keep_equal=False)

"""其中：
other：被对比的数据
align_axis=1：差异堆叠在列/行上
keep_shape=False：不保留相等的值
keep_equal=False：不保留所有原始行和列"""

In [86]:
# 数据准备
import numpy as np
import pandas as pd

df = pd.DataFrame(
    {
        "col1": ["a", "a", "b", "b", "a"],
        "col2": [1.0, 2.0, 3.0, np.nan, 5.0],
        "col3": [1.0, 2.0, 3.0, 4.0, 5.0]
    },
    columns=["col1", "col2", "col3"],
)

df
"""
df
'''
  col1  col2  col3
0    a   1.0   1.0
1    a   2.0   2.0
2    b   3.0   3.0
3    b   NaN   4.0
4    a   5.0   5.0
'''
"""

"\ndf\n'''\n  col1  col2  col3\n0    a   1.0   1.0\n1    a   2.0   2.0\n2    b   3.0   3.0\n3    b   NaN   4.0\n4    a   5.0   5.0\n'''\n"

In [89]:
# 对数据进行修改以便进行对比
df2 = df.copy()
df2.loc[0, 'col1'] = 'c'
df2.loc[2, 'col3'] = 4.0

"""
df2
'''
  col1  col2  col3
0    c   1.0   1.0
1    a   2.0   2.0
2    b   3.0   4.0
3    b   NaN   4.0
4    a   5.0   5.0
'''
"""

"\ndf2\n'''\n  col1  col2  col3\n0    c   1.0   1.0\n1    a   2.0   2.0\n2    b   3.0   4.0\n3    b   NaN   4.0\n4    a   5.0   5.0\n'''\n"

In [91]:
# 显示有差异的列
# 默认情况下，如果两个对应的值相等，它们将显示为NaN。 此外，如果整个行/列中的所有值都将从结果中省略。 其余差异将在列上对齐
df.compare(df2)

'''
  col1       col3
  self other self other
0    a     c  NaN   NaN
2  NaN   NaN  3.0   4.0
'''

'\n  col1       col3\n  self other self other\n0    a     c  NaN   NaN\n2  NaN   NaN  3.0   4.0\n'

In [93]:
# 将差异堆叠在行上：

df.compare(df2, align_axis=0)
'''
        col1  col3
0 self     a   NaN
  other    c   NaN
2 self   NaN   3.0
  other  NaN   4.0
'''


'\n        col1  col3\n0 self     a   NaN\n  other    c   NaN\n2 self   NaN   3.0\n  other  NaN   4.0\n'

In [94]:
# 保留相等的值：

df.compare(df2, keep_equal=True)
'''
  col1       col3
  self other self other
0    a     c  1.0   1.0
2    b     b  3.0   4.0
'''


'\n  col1       col3\n  self other self other\n0    a     c  1.0   1.0\n2    b     b  3.0   4.0\n'

In [ ]:
# 保留所有原始行和列：

df.compare(df2, keep_shape=True)
'''
  col1       col2       col3
  self other self other self other
0    a     c  NaN   NaN  NaN   NaN
1  NaN   NaN  NaN   NaN  NaN   NaN
2  NaN   NaN  NaN   NaN  3.0   4.0
3  NaN   NaN  NaN   NaN  NaN   NaN
4  NaN   NaN  NaN   NaN  NaN   NaN
'''


In [ ]:
# 保留所有原始行和列以及所有原始值：

df.compare(df2, keep_shape=True, keep_equal=True)
'''
  col1       col2       col3
  self other self other self other
0    a     c  1.0   1.0  1.0   1.0
1    a     a  2.0   2.0  2.0   2.0
2    b     b  3.0   3.0  3.0   4.0
3    b     b  NaN   NaN  4.0   4.0
4    a     a  5.0   5.0  5.0   5.0
'''

### equals()

In [ ]:
"""对比两个数据是否一致，测试两个对象是否包含相同的元素。

此功能允许将两个Series或DataFrame相互比较，以查看它们是否具有相同的形状和元素。 
相同位置的NaN被认为是相等的。 
列标题不必具有相同的类型，但是列中的元素必须具有相同的dtype。

此功能要求元素与其他Series或DataFrame中的元素具有相同的dtype。 但是，列标签不必具有相同的类型，只要它们仍被视为相等即可。"""

In [ ]:

df = pd.DataFrame({1: [10], 2: [20]})
"""df
    1   2
0  10  20"""

# DataFrames df和fully_equal的元素和列标签具有相同的类型和值，它们将返回True。

exactly_equal = pd.DataFrame({1: [10], 2: [20]})
"""exactly_equal
'''
    1   2
0  10  20
'''"""
df.equals(exactly_equal)
# True


In [ ]:
# DataFrames df和different_column_type具有相同的元素类型和值，但列标签具有不同的类型，它们仍将返回True。

different_column_type = pd.DataFrame({1.0: [10], 2.0: [20]})
"""different_column_type
'''
   1.0  2.0
0   10   20
'''"""
df.equals(different_column_type)
# True


In [ ]:
# DataFrames df和different_data_type为其元素的相同值具有不同的类型，即使它们的列标签具有相同的值和类型，它们也将返回False。

different_data_type = pd.DataFrame({1: [10.0], 2: [20.0]})
"""different_data_type
'''
      1     2
0  10.0  20.0
'''"""
df.equals(different_data_type)
# False

### align() 将结构不一样的DF对齐

In [ ]:
# DataFrame.align 不会组合两个数据帧，但是可以将它们对齐，以便两个数据帧具有相同的行和/或列配置

In [108]:


df1 = pd.DataFrame([[1,2,3,4], [6,7,8,9]], 
                   columns=['D', 'B', 'E', 'A'],
                   index=[1,2])

df2 = pd.DataFrame([[10,20,30,40], [60,70,80,90], [600,700,800,900]],
                   columns=['A', 'B', 'C', 'D'],
                   index=[2,3,4])

"""df1
'''
   D  B  E  A
1  1  2  3  4
2  6  7  8  9
'''"""

"""df2
'''
     A    B    C    D
2   10   20   30   40
3   60   70   80   90
4  600  700  800  900
'''"""

"df2\n'''\n     A    B    C    D\n2   10   20   30   40\n3   60   70   80   90\n4  600  700  800  900\n'''"

In [111]:
# 让我们对齐这两个数据帧，按列对齐（axis=1），并对列标签执行外部联接（join='outer'）：

a1, a2 = df1.align(df2, join='outer', axis=1)

print(a1)
"""
   A  B   C  D  E
1  4  2 NaN  1  3
2  9  7 NaN  6  8
"""

print(a2)
"""
     A    B    C    D   E
2   10   20   30   40 NaN
3   60   70   80   90 NaN
4  600  700  800  900 NaN
"""

   A  B   C  D  E
1  4  2 NaN  1  3
2  9  7 NaN  6  8
     A    B    C    D   E
2   10   20   30   40 NaN
3   60   70   80   90 NaN
4  600  700  800  900 NaN


In [ ]:
"""df1中的列已重新排列，以便与df2中的列对齐。
有一个标记为“C”的列已添加到df1，还有一个标记为“E”的列已添加到df2。这些列已用NaN填充。这是因为我们对列标签执行了外部联接。
数据帧内的所有值均未更改。
行标签未对齐；df2有第3行和第4行，而df1没有。这是因为我们要求对列进行对齐（轴=1）。"""

In [ ]:
# 如果我们在行和列上对齐，但将join参数更改为“right”,结果会不一样

a1, a2 = df1.align(df2, join='right', axis=None)
print(a1)
print(a2)
'''
     A    B   C    D
2  9.0  7.0 NaN  6.0
3  NaN  NaN NaN  NaN
4  NaN  NaN NaN  NaN

     A    B    C    D
2   10   20   30   40
3   60   70   80   90
4  600  700  800  900
'''
# 总之，如果要确保两个数据帧之间的行和/或列的排列相同，而不改变两个数据帧中包含的任何数据，请使用 DataFrame.align()。